# Velogames solver: Liège-Bastogne-Liège 2023

## Load libraries

In [72]:
using Gadfly, Velogames

## Retrieve data

Data is scraped from the [Velogames website](https://www.velogames.com/). Rider scores are the number of points they have accumulated in the current season.


In [51]:
rider_vg = getvgriders("https://www.velogames.com/spring-classics/2023/riders.php")

# keep only riders in the startlist for LBL.
rider_vg = rider_vg[rider_vg.startlist.=="#LiegeBastogneLiege", :]

rider_oneday = getpcsranking(:oneday)

Row,rank,rider,team,points,riderkey
,Int64,String,String,Int64,String
1,1,Pogačar Tadej,UAE Team Emirates,1788,aaadegjoprt
2,2,van Aert Wout,Jumbo-Visma,1560,aaenorttuvw
3,3,De Lie Arnaud,Lotto Dstny,1547,aaddeeilnru
4,4,Philipsen Jasper,Alpecin-Deceuninck,1150,aeehiijlnppprss
5,5,Laporte Christophe,Jumbo-Visma,1079,aceehhiloopprrstt
6,6,van der Poel Mathieu,Alpecin-Deceuninck,1047,aadeeehilmnoprtuv
7,7,Evenepoel Remco,Soudal - Quick Step,850,ceeeeelmnooprv
8,8,Kristoff Alexander,Uno-X Pro Cycling Team,780,aadeeffiklnorrstx
9,9,Pidcock Thomas,INEOS Grenadiers,675,accdhikmoopst


Combine datasets and calculate rider value.


In [66]:
rider_df = leftjoin(rider_vg, rider_oneday, on=:riderkey, makeunique=true)

# rename points to vgpoints
rename!(
    rider_df,
    :points => :vgpoints,
    :cost => :vgcost,
    :rank => :pcsrank,
    :points_1 => :pcspoints
)
# drop columns ending in _1
select!(rider_df, Not([:rider_1, :team_1]))

# fill in missing values in rank and points with 0
rider_df[ismissing.(rider_df.pcsrank), :pcsrank] .= 0
rider_df[ismissing.(rider_df.pcspoints), :pcspoints] .= 0
# calculate rider calc_score as the mean of the velogames score and the PCS score
rider_df[!, :calc_score] = (rider_df.pcspoints .+ rider_df.vgpoints) ./ 2
# # calculate rider value as the ratio of rider score to rider cost
rider_df[!, :calc_value] = rider_df.calc_score ./ rider_df.vgcost

rider_df

Row,,rider,team,startlist,vgpoints,vgcost,riderkey,value,pcsrank,pcspoints,calc_score,calc_value
,String,String,String,String,Int64,Int64,String,Float64,Int64?,Int64?,Float64,Float64
1,,Tadej Pogačar,UAE Team Emirates,#LiegeBastogneLiege,1858,20,aaadegjoprt,92.9,1,1788,1823.0,91.15
2,,Remco Evenepoel,Soudal - Quick Step,#LiegeBastogneLiege,520,16,ceeeeelmnooprv,32.5,7,850,685.0,42.8125
3,,Julian Alaphilippe,Soudal - Quick Step,#LiegeBastogneLiege,210,14,aaaehiiijlllnpppu,15.0,91,241,225.5,16.1071
4,,Thomas Pidcock,INEOS Grenadiers,#LiegeBastogneLiege,875,14,accdhikmoopst,62.5,9,675,775.0,55.3571
5,,Matej Mohorič,Bahrain - Victorious,#LiegeBastogneLiege,440,14,aehijmmoort,31.4286,40,407,423.5,30.25
6,,Valentin Madouas,Groupama - FDJ,#LiegeBastogneLiege,560,12,aaadeilmnnostuv,46.6667,20,543,551.5,45.9583
7,,Tiesj Benoot,Jumbo-Visma,#LiegeBastogneLiege,657,12,beeijnoostt,54.75,14,576,616.5,51.375
8,,David Gaudu,Groupama - FDJ,#LiegeBastogneLiege,0,10,aadddgiuuv,0.0,78,265,132.5,13.25
9,,Sergio Higuita,BORA - hansgrohe,#LiegeBastogneLiege,0,10,aegghiiiorstu,0.0,60,321,160.5,16.05


## Inspect the data

### Top 10 riders by points


In [67]:
sort(rider_df, :calc_score, rev=true)[1:10, :]

Row,,rider,team,startlist,vgpoints,vgcost,riderkey,value,pcsrank,pcspoints,calc_score,calc_value
,String,String,String,String,Int64,Int64,String,Float64,Int64?,Int64?,Float64,Float64
1,,Tadej Pogačar,UAE Team Emirates,#LiegeBastogneLiege,1858,20,aaadegjoprt,92.9,1,1788,1823.0,91.15
2,,Thomas Pidcock,INEOS Grenadiers,#LiegeBastogneLiege,875,14,accdhikmoopst,62.5,9,675,775.0,55.3571
3,,Ben Healy,EF Education-EasyPost,#LiegeBastogneLiege,998,4,abeehlny,249.5,22,532,765.0,191.25
4,,Remco Evenepoel,Soudal - Quick Step,#LiegeBastogneLiege,520,16,ceeeeelmnooprv,32.5,7,850,685.0,42.8125
5,,Neilson Powless,EF Education-EasyPost,#LiegeBastogneLiege,740,8,eeillnnoopsssw,92.5,12,622,681.0,85.125
6,,Tiesj Benoot,Jumbo-Visma,#LiegeBastogneLiege,657,12,beeijnoostt,54.75,14,576,616.5,51.375
7,,Mattias Skjelmose,Trek - Segafredo,#LiegeBastogneLiege,560,8,aaeeijklmmossstt,70.0,18,546,553.0,69.125
8,,Valentin Madouas,Groupama - FDJ,#LiegeBastogneLiege,560,12,aaadeilmnnostuv,46.6667,20,543,551.5,45.9583
9,,Søren Kragh Andersen,Alpecin-Deceuninck,#LiegeBastogneLiege,664,10,aadeeeghknnnrrrss,66.4,37,415,539.5,53.95


### Top 10 riders by value


In [68]:
sort(rider_df, :calc_value, rev=true)[1:10, :]

Row,,rider,team,startlist,vgpoints,vgcost,riderkey,value,pcsrank,pcspoints,calc_score,calc_value
,String,String,String,String,Int64,Int64,String,Float64,Int64?,Int64?,Float64,Float64
1,,Ben Healy,EF Education-EasyPost,#LiegeBastogneLiege,998,4,abeehlny,249.5,22,532,765.0,191.25
2,,Tadej Pogačar,UAE Team Emirates,#LiegeBastogneLiege,1858,20,aaadegjoprt,92.9,1,1788,1823.0,91.15
3,,Neilson Powless,EF Education-EasyPost,#LiegeBastogneLiege,740,8,eeillnnoopsssw,92.5,12,622,681.0,85.125
4,,Mattias Skjelmose,Trek - Segafredo,#LiegeBastogneLiege,560,8,aaeeijklmmossstt,70.0,18,546,553.0,69.125
5,,Maxim Van Gils,Lotto Dstny,#LiegeBastogneLiege,470,6,aagiilmmnsvx,78.3333,54,343,406.5,67.75
6,,Andreas Kron,Lotto Dstny,#LiegeBastogneLiege,392,6,aadeknnorrs,65.3333,42,385,388.5,64.75
7,,Andrea Bagioli,Soudal - Quick Step,#LiegeBastogneLiege,410,6,aaabdegiilnor,68.3333,51,365,387.5,64.5833
8,,Thomas Pidcock,INEOS Grenadiers,#LiegeBastogneLiege,875,14,accdhikmoopst,62.5,9,675,775.0,55.3571
9,,Søren Kragh Andersen,Alpecin-Deceuninck,#LiegeBastogneLiege,664,10,aadeeeghknnnrrrss,66.4,37,415,539.5,53.95


In [ ]:
plot(
    rider_df,
    x=:calc_score,
    y=:cost,
    Guide.xlabel("Score"),
    Guide.ylabel("Cost"),
    Guide.title("Rider value")
)

## Build the model


In [73]:
model_results = build_model_oneday(rider_df)

Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 172 cols, 344 nonzeros
2 rows, 96 cols, 170 nonzeros
2 rows, 94 cols, 166 nonzeros
Objective function is integral with scale 2

Solving MIP model with:
   2 rows
   94 cols (84 binary, 10 integer, 0 implied int., 0 continuous)
   166 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   17938.5         -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   6892.833333     6844.5             0.71%        0      0      0         3     0.0s

92.6% inactive integer columns, restarting
Model after restart has 2 rows, 7 cols (7 bin., 0 int., 0 impl., 0 cont.), and 11 nonzero

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["Tadej Pogačar", "Remco Evenepoel", "Julian Alaphilippe", "Thomas Pidcock", "Matej Mohorič", "Valentin Madouas", "Tiesj Benoot", "David Gaudu", "Sergio Higuita", "Marc Hirschi"  …  "Pau Miquel", "Yukiya Arashiro", "Liam Slock", "Ibon Ruiz", "Harry Sweeny", "Danny Van Der Tuuk", "Jorge Arcas", "Fabien Grellier", "Alan Jousseaume", "Fran Miholjević"]
And data, a 172-element Vector{Float64}:
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

## Results

### Total cost of team

Must be no more than 100.


In [75]:
# total cost
rider_df.vgcost .* model_results.data |> sum

100.0

### Selected riders


In [77]:
# selected riders
rider_df[!, :chosen] = model_results.data .|> !iszero
chosen_team = filter(:chosen => ==(true), rider_df)
chosen_team[:, [:rider, :team, :vgcost, :vgpoints, :pcspoints, :pcsrank, :calc_score, :calc_value]]

Row,rider,team,vgcost,vgpoints,pcspoints,pcsrank,calc_score,calc_value
,String,String,Int64,Int64,Int64?,Int64?,Float64,Float64
1,Tadej Pogačar,UAE Team Emirates,20,1858,1788,1,1823.0,91.15
2,Remco Evenepoel,Soudal - Quick Step,16,520,850,7,685.0,42.8125
3,Thomas Pidcock,INEOS Grenadiers,14,875,675,9,775.0,55.3571
4,Valentin Madouas,Groupama - FDJ,12,560,543,20,551.5,45.9583
5,Tiesj Benoot,Jumbo-Visma,12,657,576,14,616.5,51.375
6,Mattias Skjelmose,Trek - Segafredo,8,560,546,18,553.0,69.125
7,Neilson Powless,EF Education-EasyPost,8,740,622,12,681.0,85.125
8,Maxim Van Gils,Lotto Dstny,6,470,343,54,406.5,67.75
9,Ben Healy,EF Education-EasyPost,4,998,532,22,765.0,191.25
